In [5]:
import pandas as pd
import time
import sys
def player_data(start_year,end_year,ps=False):
    if ps == False:
        season_type='leagues'
    else:
        season_type='playoffs'
    frames=[]
    for year in range(start_year,end_year+1):
        url = 'https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_per_poss.html'
        df=pd.read_html(url)[0]
        df=df[df.Player!='Player']
        time.sleep(1)
        url2='https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_advanced.html'
        df2=pd.read_html(url2)[0]
        df2=df2[df2.Player!='Player']

        

        year_df=df.merge(df2)
        unnamed=[col for col in year_df if 'unnamed' in col.lower()]
        year_df.drop(columns=unnamed,inplace=True)
        year_df['year']=year
        frames.append(year_df)
        print(year)

        time.sleep(2)
    return pd.concat(frames)
start_year=1974
end_year=2024

#player_stats=player_data(start_year,end_year)

#player_stats.to_csv('player_stats.csv',index=False)
#player_stats.to_csv('player_stats_raw.csv',index=False)

#player_stats_ps=player_data(start_year,end_year,ps=True)
#player_stats_ps.to_csv('player_stats_ps.csv',index=False)
#player_stats_ps.to_csv('player_stats_ps_raw.csv',index=False)


In [6]:
def league_average(league='leagues'):
    time.sleep(1)
    league_url='https://www.basketball-reference.com/'+league+'/NBA_stats_per_poss.html'
    frames=pd.read_html(league_url)
    league = frames[0]
    league.columns=league.columns.droplevel()
    league=league[league.Season!='Season']

    league2 = frames[1]
    league2.columns=league2.columns.droplevel()
    league2=league2[league2.Season!='Season']
    
    
    return league,league2
league,league2 = league_average()
league.to_csv('league_average.csv',index=False)

league2.to_csv('league_average_ps.csv',index=False)



In [7]:
import unicodedata
import re
def transliterate(text):
    # Normalize to NFKD form (separate characters from diacritics)
    text = unicodedata.normalize('NFKD', text)
    # Remove diacritics
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    # Replace remaining non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text
def improved_transliterate(text):
    # Custom replacements
    custom_replacements = {
        'ö': 'o', 'Ö': 'O',
        'ä': 'a', 'Ä': 'A',
        'ü': 'u', 'Ü': 'U',
        'ß': 'ss',
        # Add more custom replacements as needed
    }
    
    for original, replacement in custom_replacements.items():
        text = text.replace(original, replacement)
    
    # Normalize to NFD form (separate characters from diacritics)
    text = unicodedata.normalize('NFD', text)
    # Remove diacritics
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    # Replace remaining non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

In [15]:
players = pd.read_csv('player_stats_raw.csv')
print(len(players))
players.rename(columns={'Player':'player','Tm':'team'},inplace=True)
players['player']=players['player'].str.strip('*')
#players['player']=players['player'].str.encode('utf-8')

#players['player']=players['player'].apply(improved_transliterate)
player_index=pd.read_csv('index_master.csv')
#player_index['player']=player_index['player'].str.encode('utf-8')
#player_index['player']=player_index['player'].astype(str)
#player_index['player']=player_index['player'].apply(improved_transliterate)

players=players.merge(player_index,on=['player','team','year'])
players.drop_duplicates(inplace=True)
print(len(players))
players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch01')|(players.Pos!='PF')|(players.year!=1989)].reset_index(drop=True)

players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch02')|(players.Pos!='C')|(players.year!=1989)].reset_index(drop=True)
players
players.to_csv('player_stats.csv',index=False)


25751
25753


In [16]:
len(players)

25751

In [17]:
'''
cloned=[]
for year in range(1974,2025):
    year_df=players[players.year==year].reset_index(drop=True)
    year_df['player_tm']=year_df['player']+year_df['team']
    vc=year_df['player_tm'].value_counts()
    
    vc=vc[vc>1]
    cloned.extend(vc.index)
    print(year)
    print(vc)
print(set(cloned))
'''

"\ncloned=[]\nfor year in range(1974,2025):\n    year_df=players[players.year==year].reset_index(drop=True)\n    year_df['player_tm']=year_df['player']+year_df['team']\n    vc=year_df['player_tm'].value_counts()\n    \n    vc=vc[vc>1]\n    cloned.extend(vc.index)\n    print(year)\n    print(vc)\nprint(set(cloned))\n"

In [19]:
players = pd.read_csv('player_stats_ps_raw.csv')
print(len(players))
players.rename(columns={'Player':'player','Tm':'team'},inplace=True)
players['player']=players['player'].str.strip('*')
#players['player']=players['player'].str.encode('utf-8')

#players['player']=players['player'].apply(improved_transliterate)
player_index=pd.read_csv('index_master.csv')
#player_index['player']=player_index['player'].str.encode('utf-8')
#player_index['player']=player_index['player'].astype(str)
#player_index['player']=player_index['player'].apply(improved_transliterate)

players=players.merge(player_index,on=['player','team','year'],how='left')
players.drop_duplicates(inplace=True)
print(len(players))
players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch01')|(players.Pos!='PF')|(players.year!=1989)].reset_index(drop=True)

players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch02')|(players.Pos!='C')|(players.year!=1989)].reset_index(drop=True)
players


9304
9304


,Rk,player,Pos,Age,team,G,GS,MP,FG,FGA,...,WS,WS/48,OBPM,DBPM,BPM,VORP,year,url,bref_id,nba_id
0,1,Kareem Abdul-Jabbar,C,26,MIL,16,NaN,758,14.4,25.8,...,4.7,0.297,8.3,1.6,9.9,2.2,1974,https://www.basketball-reference.com/players/a...,abdulka01,76003.0
1,2,Don Adams,SF,26,DET,7,NaN,256,5.5,14.4,...,0.1,0.026,-1.0,-0.2,-1.2,0.1,1974,https://www.basketball-reference.com/players/a...,adamsdo01,76012.0
2,3,Rick Adelman,PG,27,CHI,9,NaN,108,7.3,15.6,...,0.2,0.074,-1.0,2.1,1.2,0.1,1974,https://www.basketball-reference.com/players/a...,adelmri01,76015.0
3,4,Dennis Awtrey,C,25,CHI,10,NaN,158,3.8,8.8,...,0.1,0.034,-2.6,1.6,-1.0,0.0,1974,https://www.basketball-reference.com/players/a...,awtrede01,76078.0
4,5,Henry Bibby,PG,24,NYK,10,0.0,89,8.7,24.5,...,-0.1,-0.035,-2.6,-1.3,-3.9,0.0,1974,https://www.basketball-reference.com/players/b...,bibbyhe01,76162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9299,210,Jaylin Williams,C,21,OKC,10,0.0,127,6.4,13.2,...,0.6,0.214,2.4,4.1,6.5,0.3,2024,https://www.basketball-reference.com/players/w...,willija07,1631119.0
9300,211,Kenrich Williams,PF,29,OKC,7,0.0,32,3.2,12.7,...,0.0,0.023,-6.5,1.8,-4.7,0.0,2024,https://www.basketball-reference.com/players/w...,willike04,1629026.0
9301,212,Delon Wright,PG,31,MIA,4,1.0,107,5.9,9.9,...,0.4,0.158,0.1,3.8,3.9,0.2,2024,https://www.basketball-reference.com/players/w...,wrighde01,1626153.0
9302,213,Thaddeus Young,PF,35,PHO,1,0.0,4,0.0,0.0,...,0.0,-0.073,-3.2,-3.1,-6.3,0.0,2024,https://www.basketball-reference.com/players/y...,youngth01,201152.0


In [32]:
#players.to_csv('player_stats_ps.csv',index=False)
missing_id=players[players.nba_id.isna()].reset_index(drop=True)
search=pd.read_csv('playersearch_index.csv')
missing_id.drop(columns=['url','bref_id','nba_id'],inplace=True)
temp_df=missing_id.merge(search,how='left').reset_index(drop=True)


luca=temp_df.loc[temp_df['player']=='Luca Vildoza'].reset_index(drop=True)
luca['url']=['https://www.basketball-reference.com/players/v/vildolu01.html']
luca['bref_id']=['vildolu01']
luca['nba_id']=[1630492.0]
players = pd.concat([players,temp_df,luca])
players=players.dropna(subset='nba_id')
players

,Rk,player,Pos,Age,team,G,GS,MP,FG,FGA,...,WS,WS/48,OBPM,DBPM,BPM,VORP,year,url,bref_id,nba_id
0,1,Kareem Abdul-Jabbar,C,26,MIL,16,NaN,758,14.4,25.8,...,4.7,0.297,8.3,1.6,9.9,2.2,1974,https://www.basketball-reference.com/players/a...,abdulka01,76003.0
1,2,Don Adams,SF,26,DET,7,NaN,256,5.5,14.4,...,0.1,0.026,-1.0,-0.2,-1.2,0.1,1974,https://www.basketball-reference.com/players/a...,adamsdo01,76012.0
2,3,Rick Adelman,PG,27,CHI,9,NaN,108,7.3,15.6,...,0.2,0.074,-1.0,2.1,1.2,0.1,1974,https://www.basketball-reference.com/players/a...,adelmri01,76015.0
3,4,Dennis Awtrey,C,25,CHI,10,NaN,158,3.8,8.8,...,0.1,0.034,-2.6,1.6,-1.0,0.0,1974,https://www.basketball-reference.com/players/a...,awtrede01,76078.0
4,5,Henry Bibby,PG,24,NYK,10,0.0,89,8.7,24.5,...,-0.1,-0.035,-2.6,-1.3,-3.9,0.0,1974,https://www.basketball-reference.com/players/b...,bibbyhe01,76162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,1,Jaylen Adams,PG,23,POR,3,0.0,21,6.7,20.2,...,0.0,-0.051,-3.6,-2.6,-6.1,0.0,2020,https://www.basketball-reference.com/players/a...,adamsja01,1629121.0
12,79,Shaquille Harrison,SG,29,LAL,8,0.0,28,7.0,12.2,...,0.2,0.286,0.7,2.1,2.8,0.0,2023,https://www.basketball-reference.com/players/h...,harrish01,1627885.0
13,97,DaQuan Jeffries,SG,25,NYK,2,0.0,5,0.0,0.0,...,0.0,0.030,-19.0,-1.0,-20.1,0.0,2023,https://www.basketball-reference.com/players/j...,jeffrda01,1629610.0
14,195,Tristan Thompson,C,31,LAL,6,0.0,32,7.6,16.8,...,0.0,0.003,-3.8,-3.0,-6.8,0.0,2023,https://www.basketball-reference.com/players/t...,thomptr01,202684.0


In [33]:
players.to_csv('player_stats_ps.csv',index=False)